<a href="https://colab.research.google.com/github/arnabmukho/RNA_Seq_Data_Analysis/blob/main/RNA_Seq_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reload_ext rpy2.ipython


Mount your Google drive to the notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# RNA Sequencing Data Analysis

Install following softwares

In [ ]:
!apt-get install fastqc
!apt-get install bwa
!apt-get install samtools
!apt-get install bedtools

1. Checking raw read quality

The first thing one should do is to check the quality of reads.

Sequence read quality can be checked using ‘FASTQC’ software. The software can be downloaded from the following link: https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip This will download the zipped file. Extract the content of the file (Right-click on the file > extract here). The software will be usable just like any GUI software.

In the command line or google colab, fastqc can be downloaded as instructed in the ‘Installing Softwares’ section.

To run a quality check, run the following command in the cell after installation: !fastqc p/sample.fastq -o output_dir

(Note: Change the directory path, file name and output directory name accordingly)

This will result in a '.html' file in the output directory (out_dir) mentioned.

In [ ]:
!fastqc /content/drive/MyDrive/RNA_seq_V/fastq_dir/sample.fastq -o /content/drive/MyDrive/RNA_seq_V/fastq_dir

Specified output directory '/content/drive/MyDrive/RNA_seq_V/fastq_dir' does not exist


**2. Removing adapters (Not mandatory in mRNA DGE analysis)**

Adapters can be trimmed using the following command:
! cutadapt -a adapter_sequence -o path/sample_output.fastq path/sample_input.fastq

Adapter sequences contaminate reads content in sequencing and can result in misalignment and thus increase in the number of unaligned reads.

Since the number of reads in mRNA sequencing for DGE analysis is huge, this contamination is expected to be small, thus this step is not required for the analysis.


**3. Indexing the reference genome**

Just like an index in a book is needed for a quick search inside the book, Indexing the reference file allows the software to search and align the reads to the reference genome which saves both memory as well as time.

To index the genome, run the following command to index the genome:
!bwa index -a is path/reference_file

Here, the reference file is the DNA fasta file in ‘.fna’ format.
The above command will generate 5 index files.
The above files should be kept in the same folder as the reference genome and should NOT be renamed.


In [ ]:
!bwa index -a is /content/drive/MyDrive/RNA_seq_V/reference_dir/sequence.fasta

**4. Aligning raw reads against the indexed reference genome**


Once the genome is indexed, the reads (in sample.fastq) are aligned and sequence alignment (SA) coordinates are obtained.

To align the reads, run the following command:

! bwa aln -q 20 path/reference_file sample.fastq > path/sample.sai (**Note:** Change the directory path and file name accordingly)

The -q option is used to filter reads based on the average read quality score (mean Phred score).

Phred score threshold of 20 (-q 20) means that on average, the likelihood of finding an incorrect base in the read is 1 in 100.


In [ ]:
! bwa aln -q 20 /content/drive/MyDrive/RNA_seq_V/reference_dir/sequence.fasta /content/drive/MyDrive/RNA_seq_V/fastq_dir/sample.fastq > /content/drive/MyDrive/RNA_seq_V/sai_dir/sample.sai

**5. Sai to sam conversion**

This step generates the alignment in SAM format, a human-readable format where each line represents a mapped read.

Run the following command to create a sam file:
!bwa samse path/reference_file path/sample.sai path/sample.fastq > path/sample.sam (**Note:** Change the directory path and file name accordingly)

The samse option is used for the single-end (se) reads. For paired-end (pe), ‘sampe’ option is used.

SAM file contains the following:

**Column 1:** Name of the read.

**Column 2:** Mapping information about the read denoted as Flag integer. Each value has different mapping information associated with it. For example, a score of 16 means the read is aligned in the reverse direction. For mapped and unmapped reads the Flag value is 2 and 4 respectively.

**Column 3:** ID of the reference genome.

**Column 4:** 1-based Left-most position (wrt. Reference sequence) of the mapped read.

**Column 5:** Mapping quality as Phred score. It gives the information about the probability that the read is mapped correctly on the reference. For example, a Phred score of 30 means that the probability of incorrect mapping of the read is 1 in 1000.

**Column 6:** CIGAR, an alphanumeric combination which gives the information about how the read is mapped to the reference in terms of insertion, deletion, matches and mismatches. For example, 6M2I4M means 6 bases in the read matches to the reference,  2 bases present in reads that are absent in reference and 4 bases in the read that match with the reference.

**Column 7:** RNEXT provides the name of the reference for the next read. If information is unavailable, it will contain “*”, in case of identical reference, it will be “=”.

**Column 8:** PNEXT, the 1-based position of the next read. It will be 0 if information is unavailable.

**Column 9:** TLEN, length of the reference template to which the read is mapping. For instance, in case of 100% match between the read and the template this value will be equal to read length, whereas in case of deletion in the read this value will be more than the read length.

**Column 10:** Sequence of the read. (SEQ)

**Column 11:** Quality of the read in ASCII format, similar to the one provided in the Fastq sequence file
For more information on the sam file, please refer to: https://samtools.github.io/hts-specs/SAMv1.pdf





In [ ]:
!bwa samse /content/drive/MyDrive/RNA_seq_V/reference_dir/sequence.fasta /content/drive/MyDrive/RNA_seq_V/sai_dir/sample.sai /content/drive/MyDrive/RNA_seq_V/fastq_dir/sample.fastq > /content/drive/MyDrive/RNA_seq_V/sam_dir/sample.sam

let's visualise sam file

In [ ]:
! head -15 /content/drive/MyDrive/RNA_seq_V/sam_dir/sample.sam

**6. Converting .sam to .bam file**

SAM file is converted to its binary format (BAM) using the following command:

!samtools view -q1 -Sb path/sample.sam > path/sample.bam (**Note:** Change the directory path and file name accordingly)

The -q option applies filter on mapping quality (MAPQ) of the reads, where 0 means that all reads will be considered and converted to bam whereas a higher value denotes that more uniquely mapped reads will be selected from the bam file.

In [ ]:
!samtools view -q1 -Sb /content/drive/MyDrive/RNA_seq_V/sam_dir/sample.sam > /content/drive/MyDrive/RNA_seq_V/bam_dir/sample.bam

**7. Sorting the .bam file**

The bam file is sorted based on the position of the read for follow-up steps using following command:
!samtools sort path/sample.bam  -o path/sample.sorted.bam


In [ ]:
!samtools sort /content/drive/MyDrive/RNA_seq_V/bam_dir/sample.bam -o /content/drive/MyDrive/RNA_seq_V/bam_dir/sample.sorted.bam

**8. Indexing the sorted bam file**

Similar to indexing the reference genome in the initial steps, the sorted bam file is indexed for fast implementation of the following steps.


In [ ]:
!samtools index /content/drive/MyDrive/RNA_seq_V/bam_dir/sample.sorted.bam

**9. Displaying statistics of the mapped reads**

To check the total number and proportion of  mapped reads, run:

!samtools flagstat path/sample.sorted.bam > path/sample.flagstat

(**Note:** Change the directory path and file name accordingly)

After obtaining the statistics on mapped reads, and information on genome size and read length, genome coverage should be calculated using the following formula:

Coverage = (Number of reads mapped * Length of a read) / Effective genome size

The effective genome size is the size of the genome providing reads, for DNA sequencing the effective genome size is the actual genome size whereas exome sequencing in the effective genome size is the sum of the exonic regions.

Sample with good coverage (>10X for DGE analysis) only should be proceeded with the follow-up analysis.


In [ ]:
!samtools flagstat /content/drive/MyDrive/RNA_seq/bam_dir/sample.sorted.bam > /content/drive/MyDrive/RNA_seq/flagstat_dir/sample.flagstat

10. Converting Bam to Bed file

The .bam file should be converted to .bed format to calculate the coverage per gene.

Run to following command:

!bedtools bamtobed -i path/sample.sorted.bam > path/sample.bed (Note: Change the directory path and file name accordingly)

The bed file contains:

Column 1: Name of the Chromosome/contig.

Column 2: 1 based start position on the chromosome/contig to which a read has mapped.

Column 3: End position on the chromosome/contig to which a read has mapped.

Column 4: Name of the read mapped.

Column 5: Average Phred score of the read (average of Phred scores from the fastq file).

Column 6: Strand of the chromosome/contig to which the read has mapped.

In [ ]:
!bedtools bamtobed -i /content/drive/MyDrive/RNA_seq/bam_dir/sample.sorted.bam > /content/drive/MyDrive/RNA_seq/bed_dir/sample.bed

11. Preparing annotation bed files

To obtain the gene coverage, an annotation file with gene name, position, strand information, and description is needed in .bed format.

To convert GFF3 file to .bed format, run:

!python path/gff2bed.py path/annotation.gff3 path/annotation.bed

(Note: Change the directory path and file name accordingly)

In [ ]:
!python  /content/drive/MyDrive/RNA_seq/annotation_dir/gff2bed.py /content/drive/MyDrive/RNA_seq/annotation_dir/sequence.gff3 /content/drive/MyDrive/RNA_seq/annotation_dir/sequence.bed

**12. Generating gene coverage file**

To generate a file containing how many reads have mapped on each gene of the genome, run:

!bedtools coverage -s/-S  -a path/annotation.bed -b path/sample.bed > path/sample.cov

-s (forward strand sequencing) or -S (reverse strand sequencing) option should be used depending on the sequencing methodology used.

If sequencing is not strand specific or paired-end, then -s/S option is not needed.

The coverage file (.cov) contains:

**Column 1:** Name of the Chromosome/contig.

**Column 2:** 0-based start position of the gene.

**Column 3:** End position of the gene

**Column 4:** ID of the the gene

**Column 5:** Name of the gene

**Column 6:** Strand of the gene

**Column 7:** Number of reads mapping to the gene

**Column 8** Length of gene covered by the reads

**Column 9:** Length of the gene

**Column 10:** coverage of gene in terms of fraction (col. 8 divided by col. 9)

In [ ]:
!bedtools coverage -S  -a /content/drive/MyDrive/RNA_seq/annotation_dir/sequence.bed -b /content/drive/MyDrive/RNA_seq/bed_dir/sample.bed > /content/drive/MyDrive/RNA_seq/coverage_dir/sample.cov

In [ ]:
! head /content/drive/MyDrive/RNA_seq/coverage_dir/sample.cov

head: cannot open '/content/drive/MyDrive/RNA_seq/coverage_dir/sample.cov' for reading: No such file or directory


In [ ]:
%reload_ext rpy2.ipython

Differential Gene Expression

Download and install the following packages

In [ ]:
%%R
install.packages('BiocManager')
BiocManager::install("edgeR")
install.packages('statmod')
install.packages('gplots')

Load the installed packages

In [ ]:
%%R
library("edgeR")
library("statmod")
library("gplots")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Read the coverage files and make one gene count matrix. (Run following commands in one cell)

In [ ]:
%%R
GenewiseCounts <- c()
Gene_coverage <- c()
X <- c()
index= 0
list_all = list.files("/content/drive/MyDrive/RNAseq/RNA_seq/coverage_files/")
for (filename in list_all) {
index = index +1
A <- read.table(paste("/content/drive/MyDrive/RNAseq/RNA_seq/coverage_files/",filename, sep="/"),header = F)
A <- A[order(A$V5),]
Genes <- A$V5
gene_name <- A$V4
GenewiseCounts <- cbind(GenewiseCounts,A$V7)
colnames(GenewiseCounts)[index]= filename
}

Display the first 5 lines of the gene count matrix (GenewiseCounts) using head() function

In [ ]:
%%R
head(GenewiseCounts)

Name rows as genes and columns of the read count matrix / coverage file as sample names

In [ ]:
%%R
rownames(GenewiseCounts) <- Genes
colnames(GenewiseCounts) <- c(
paste(rep("WT_UT",3),seq(1,3),sep = "_"),
paste(rep("WT_NO",3),seq(1,3),sep = "_"),
paste(rep("KO_UT",2),seq(1,2),sep = "_"),
paste(rep("KO_NO",3),seq(1,3),sep = "_"))

**Remove reads mapping ribosomal RNA (rRNA) features**
In the following code, change the rrna gene names according to the genome that you have.

In case of eukaryotic mRNA sequencing where poly-A enrichment is done during library preparation, skip this step

In [ ]:
%%R
rrna <- c("rrs","rrl","rrf")
mrna_mapped_reads <- (colSums(GenewiseCounts)-colSums(GenewiseCounts[which(rownames(GenewiseCounts)%in% rrna),]))/1000000
print(mrna_mapped_reads)
GenewiseCounts <- GenewiseCounts[-which(rownames(GenewiseCounts)%in% rrna),]

**Create sample groups (WT and KO)**


In [ ]:
%%R
group<- c(rep("WT_UT",3),rep("WT_NO",3),rep("KO_UT",2),rep("KO_NO",3))
print(group)

The edgeR package stores data in a simple list-based data object called a DGEList. The main components of a DGEList object are a matrix of read counts, sample information in the data.frame format and optional gene annotation. We enter the counts into a DGEList object using the function DGEList:

In [ ]:
%%R
y<-DGEList(GenewiseCounts,group=group)
y$samples

The expression profiles of individual samples can be explored more closely with mean-difference (MD) plots. An MD plot visualizes the library size-adjusted log-fold change between two libraries (the difference) against the average log-expression across those libraries (the mean). The following command produces an MD plot that compares sample 1 to an artificial reference library constructed from the average of all the other samples:

In [ ]:
%%R
plotMD(y,column=1)
abline(h=0,col="red",lty=2,lwd=2)

**Filter to remove low counts**

Genes that have very low counts across all the libraries should be removed prior to downstream analysis. From biological point of view, a gene must be expressed at some minimal level before it is likely to be translated into a protein or to be considered biologically important.

From a statistical point of view, genes with consistently low counts are very unlikely be assessed as significantly DE. As a rule of thumb, we require that a gene have a count of at least 10–15 before it is considered to be expressed in the study. Here, we are filtering out genes with read count less than 10 reads.


In [ ]:
%%R
keep <- rowSums(y$counts >= 10) >= ncol(GenewiseCounts)
table(keep)
y<-y[keep,keep.lib.sizes=FALSE]

**Normalize for composition bias**

Normalization by trimmed mean of M values (TMM) is performed by using the calcNormFactors function. It calculates a set of normalization factors, one for each sample, to eliminate composition biases between libraries. The product of these factors and the library sizes defines the effective library size, which replaces the original library size in all downstream analyses.

In [ ]:
%%R
y<-calcNormFactors(y)
y$samples

**Explore differences between samples**

The RNA samples can be clustered in two dimensions using multi-dimensional scaling (MDS) plots. This is both an analysis step and a quality control step to explore the overall differences between the expression profiles of the different samples.

In [ ]:
%%R
pch<- c(rep(25,3),rep(6,3),rep(10,2),rep(20,3))
colors<- c(rep("red",3),rep("blue",3),rep("orange",2),rep("black",3))
plotMDS(y,col=colors,pch = pch)
legend("top", legend=colnames(GenewiseCounts), pch=pch, col=colors,ncol = 1,cex = 0.8)

Check the expression profiles of individual samples after normalisation by plotting MD plot

In [ ]:
%%R
plotMD(y,column=1)
abline(h=0,col="red",lty=2,lwd=2)

Linear modeling and differential expression analysis in edgeR requires a design matrix to be specified. The design matrix defines how the experimental effects are parametrized in the linear models.

In [ ]:
%%R
design <- model.matrix(~0+group)
design

**Dispersion estimation**

edgeR uses the negative binomial (NB) distribution to model the read counts for each gene in each sample. The dispersion parameter of the NB distribution accounts for variability between biological replicates. edgeR estimates an empirical Bayes moderated dispersion for each individual gene. It also estimates a common dispersion, which is a global dispersion estimate averaged over all genes, and a trended dispersion where the dispersion of a gene is predicted from its abundance.
Here robust=TRUE has been used to protect the empirical Bayes estimates against the possibility of outlier genes with exceptionally large or small individual dispersions.
The dispersion estimates can be visualized with plotBCV function.

In [ ]:
%%R
y<-estimateDisp(y,design,robust=TRUE)
plotBCV(y)

**Estimating quasi-likelihood (QL) dispersions**

For RNA-seq studies, the NB dispersions tend to be higher for genes with very low counts. The dispersion trend tends to decrease smoothly with abundance and to asymptotic to a constant value for genes with larger counts. The NB model can be extended with quasi-likelihood (QL) methods to account for gene-specific variability from both biological and technical sources. Under the QL framework, the NB dispersion trend is used to describe the overall biological variability across all genes, and gene-specific variability above and below the overall level is picked up by the QL dispersion.

In [ ]:
%%R
fit<-glmQLFit(y,design,robust=TRUE)
plotQLDisp(fit)

**Heatmap clustering**

Heatmaps are a popular way to display differential expression results. To create a heatmap, we first convert the read counts into log2-counts-per-million (logCPM) values.

In [ ]:
%%R
logCPM<-cpm(y,log=TRUE)
head(logCPM)
t_logCPM<-t(scale(t(logCPM)))
col.pan<-colorpanel(100,"green","black","red")
heatmap.2(t_logCPM, col=col.pan, Rowv=TRUE, scale="none", trace="none", dendrogram="column",
          labRow = F,cexCol=1.4, symkey = F,key.par = list(cex=0.5), symm=F,symbreaks = FALSE,density.info="none",
          margin=c(10,9),lhei=c(2,10), lwid=c(2,6),key = TRUE, keysize = 2)

**Preparing comparative groups and Analysing pairwise differential expression**

The differential expression analysis comparing two groups can be easily extended to comparisons between three or more groups. This is done by creating a matrix of independent contrasts.
Suppose we want to compare the four groups, an appropriate contrast matrix can be created using the makeContrasts() function as shown below, to make pairwise comparisons between all four groups.
The QL F-test is then applied to identify genes that are DE between the four groups. This combines the four pairwise comparisons into a single F-statistic and p-value. The top set of significant genes can be displayed with topTags.

In [ ]:
%%R
comparisons<-c("groupKO_UT-groupWT_UT")
x = comparisons[1] 			#for first comparative group
mvsw<-makeContrasts(x,levels=design)
res<-glmQLFTest(fit,contrast=mvsw)
W<- topTags(res,n = nrow(res$table))

Write the output to a file

In [ ]:
%%R
write.csv(W, "/content/drive/MyDrive/RNA_seq/KOvsWT.csv")

**Generating a Volcano plot**

Volcano plots provide an effective means for visualizing the direction, magnitude, and significance of changes in gene expression.

In [ ]:
%%R
with(W$table, plot(logFC, -log10(PValue), pch=20, main="Volcano plot",
                   col= ifelse((FDR <= 0.05 & abs(logFC) >= log2(2)),"red","black")))
legend("topleft", legend=c("|logFC| >= log2(2) & pvalue<= 0.05","|logFC| < log2(2) or pvalue >0.05"),
       pch=c(20,20), col=c("red","black"), ncol = 1,cex = 0.8)